In [1]:
import numpy as np
import pandas as pd
import ray
from configs.load_specific_data import pharma_basket
from env.multi_agent.worker_hrl import Worker
from env.multi_agent.hrl import HRL
from ray.rllib.utils import check_env
from ray.train.rl import RLTrainer
from ray.air.config import RunConfig, ScalingConfig
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.rllib.algorithms.bc.bc import BC
from ray.rllib.algorithms import a2c
from ray.tune.registry import register_env
import gymnasium
ray.init(_temp_dir='/Users/floriankockler/rayresults/')
import warnings
warnings.simplefilter(action='ignore', category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import os
os.environ['PYTHONWARNINGS'] = 'ignore'

2023-09-06 18:47:57,645	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-09-06 18:48:00,624	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [2]:
train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()
# train_df = train_df[train_df["tic"] == "PFE.US"]
# validate_df = validate_df[validate_df["tic"] == "PFE.US"]


In [ ]:

manager_config = {
    "df": train_df,

}
config={
        "manager_config": manager_config
        }

env = HRL(config)

n_iterations = 7000

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [3]:
import os 
manager_config = {
    "df": train_df,

}
config={
        "manager_config": manager_config
        }

env = HRL(config)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hierarch_env", env_creator)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 

first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 
manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }


trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 100},local_dir="/Users/floriankockler/rayresults/trainingtest1"),
    scaling_config=ScalingConfig(
        num_workers=1,
        use_gpu=False,
    ),
 
    algorithm=a2c.A2C,
    config={
        "multiagent": {
            "policies": {
                "worker_policy": worker_policy_spec,
                "manager_policy": manager_policy_spec,
            },
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env": "hierarch_env",
        "env_config": hrl_config,
        "framework": "tf2",
        "evaluation_num_workers": 5,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},
    },
)
result = trainer.fit()

2023-09-06 18:48:03,162	WARNING worker.py:2006 -- Warning: The actor AIRRLTrainer is very large (26 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=34492) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(AIRA2C pid=34492) 2023-09-06 18:48:09,414	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(AIRA2C pid=34492) 2023-09-06 18:48:09,414	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(AIRA2C pid=34492) 2023-09-06 18:48:09,414	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=34502) DeprecationWarning

(RolloutWorker pid=34524) =========HRL is done=============
(RolloutWorker pid=34524) HRL is done
(RolloutWorker pid=34524) day: 6304, episode: 2
(RolloutWorker pid=34524) Total Cash Transfers: 60
(RolloutWorker pid=34524) total_portfolio_trades: 49328.0
(RolloutWorker pid=34524) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=34524) End_Portfolio_Value: 7681662.0
(RolloutWorker pid=34524) Annual Return: 8.10 %
(RolloutWorker pid=34524) Worker ID: ABT.US Current Stock Exposure: 0
(RolloutWorker pid=34524) Worker ID: AMGN.US Current Stock Exposure: 219005
(RolloutWorker pid=34524) Worker ID: BDX.US Current Stock Exposure: 858463
(RolloutWorker pid=34524) Worker ID: BMY.US Current Stock Exposure: 433827
(RolloutWorker pid=34524) Worker ID: HUM.US Current Stock Exposure: 421826
(RolloutWorker pid=34524) Worker ID: JNJ.US Current Stock Exposure: 36554
(RolloutWorker pid=34524) Worker ID: LLY.US Current Stock Exposure: 93683
(RolloutWorker pid=34524) Worker ID: MDT.US Current Stock Expos

(RolloutWorker pid=34521) 2023-09-06 18:51:47,042	WARNING env_runner_v2.py:155 -- More than 88270 observations in 6305 env steps for episode 251102852791450314 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.
(AIRA2C pid=34492) 2023-09-06 18:54:58,594	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration

(RolloutWorker pid=34522) =========HRL is done============= [repeated 5x across cluster]
(RolloutWorker pid=34522) HRL is done [repeated 5x across cluster]
(RolloutWorker pid=34522) day: 6304, episode: 3 [repeated 5x across cluster]
(RolloutWorker pid=34522) Total Cash Transfers: 52 [repeated 5x across cluster]
(RolloutWorker pid=34522) total_portfolio_trades: 48317.0 [repeated 5x across cluster]
(RolloutWorker pid=34522) Beginn_Portfolio_Value: 2000000 [repeated 5x across cluster]
(RolloutWorker pid=34522) End_Portfolio_Value: 7598908.5 [repeated 5x across cluster]
(RolloutWorker pid=34522) Annual Return: 8.03 % [repeated 5x across cluster]
(RolloutWorker pid=34522) Worker ID: ABT.US Current Stock Exposure: 84903 [repeated 5x across cluster]
(RolloutWorker pid=34522) Worker ID: AMGN.US Current Stock Exposure: 2403055 [repeated 5x across cluster]
(RolloutWorker pid=34522) Worker ID: BDX.US Current Stock Exposure: 265998 [repeated 5x across cluster]
(RolloutWorker pid=34522) Worker ID: 

(AIRA2C pid=34492) 2023-09-06 18:58:22,559	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').


...

2023-09-06 20:36:57,832	WARNING util.py:315 -- The `process_trial_result` operation took 0.559 s, which may be a performance bottleneck.
(AIRA2C pid=34492) 2023-09-06 20:40:56,129	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').
2023-09-06 20:42:14,000	WARNING util.py:315 -- The `process_trial_result` operation took 0.602 s, which may be a performance bottleneck.
(AIRA2C pid=34492) 2023-09-06 20:46:28,388	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `ev

(RolloutWorker pid=34521) =========HRL is done============= [repeated 2x across cluster]
(RolloutWorker pid=34521) HRL is done [repeated 2x across cluster]
(RolloutWorker pid=34521) day: 6304, episode: 33 [repeated 2x across cluster]
(RolloutWorker pid=34521) Total Cash Transfers: 30 [repeated 2x across cluster]
(RolloutWorker pid=34521) total_portfolio_trades: 60251.0 [repeated 2x across cluster]
(RolloutWorker pid=34521) Beginn_Portfolio_Value: 2000000 [repeated 2x across cluster]
(RolloutWorker pid=34521) End_Portfolio_Value: 63964432.0 [repeated 2x across cluster]
(RolloutWorker pid=34521) Annual Return: 22.21 % [repeated 2x across cluster]
(RolloutWorker pid=34521) Worker ID: ABT.US Current Stock Exposure: 4267427 [repeated 2x across cluster]
(RolloutWorker pid=34521) Worker ID: AMGN.US Current Stock Exposure: 0 [repeated 2x across cluster]
(RolloutWorker pid=34521) Worker ID: BDX.US Current Stock Exposure: 1218780 [repeated 2x across cluster]
(RolloutWorker pid=34521) Worker ID: 

(AIRA2C pid=34492) 2023-09-06 20:51:50,111	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').
(AIRA2C pid=34492) 2023-09-06 20:57:19,105	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').
2023-09-06 20:58:44,259	WARNING util.py:315 -- The `process_trial_result` operation took 0.541 s, which may be a performance bottleneck.


(RolloutWorker pid=34521) =========HRL is done=============
(RolloutWorker pid=34521) HRL is done
(RolloutWorker pid=34521) day: 6304, episode: 34
(RolloutWorker pid=34521) Total Cash Transfers: 24
(RolloutWorker pid=34521) total_portfolio_trades: 60213.0
(RolloutWorker pid=34521) Beginn_Portfolio_Value: 2000000
(RolloutWorker pid=34521) End_Portfolio_Value: 59368540.0
(RolloutWorker pid=34521) Annual Return: 21.69 %
(RolloutWorker pid=34521) Worker ID: ABT.US Current Stock Exposure: 2366515
(RolloutWorker pid=34521) Worker ID: AMGN.US Current Stock Exposure: 13822830
(RolloutWorker pid=34521) Worker ID: BDX.US Current Stock Exposure: 3620598
(RolloutWorker pid=34521) Worker ID: BMY.US Current Stock Exposure: 1081811
(RolloutWorker pid=34521) Worker ID: HUM.US Current Stock Exposure: 1682572
(RolloutWorker pid=34521) Worker ID: JNJ.US Current Stock Exposure: 2637619
(RolloutWorker pid=34521) Worker ID: LLY.US Current Stock Exposure: 1147929
(RolloutWorker pid=34521) Worker ID: MDT.US C

(AIRA2C pid=34492) 2023-09-06 21:02:48,847	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').
(AIRA2C pid=34492) 2023-09-06 21:08:51,796	WARNING algorithm.py:1061 -- Calling `sample()` on your remote evaluation worker(s) resulted in a timeout (after the configured 180.0 seconds)! Try to set `evaluation_sample_timeout_s` in your config to a larger value. If your episodes don't terminate easily, you may also want to set `evaluation_duration_unit` to 'timesteps' (instead of 'episodes').
2023-09-06 21:10:53,958	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 0.507 s, which may be a performance bottleneck.
2023-09-06 21:10:54,065	WARNING util.py:315 -- The `process_trial_re

In [ ]:
train_df.columns

Index(['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14'],
      dtype='object')

In [ ]:
# worker_df = train_df[train_df["tic"] == "PFE.US"]

# config={"df": worker_df}
        
# env = Worker(env_config=config)
# check_env(env)



# n_iterations = 500

# state = env.reset()

# for _ in range(n_iterations):

#     action = env.action_space.sample()

#     obs, reward, done, truncated, info= env.step(action)

#     # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
#     if done:
#         print("Episode finished!")
#         state = env.reset()
#     else:
#         state = obs

In [ ]:
manager_config = {
    "df": train_df,

}
config={



        "manager_config": manager_config
        }
env = HRL(env_config=config)
check_env(env)

In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    "rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/falsetry1", progress_reporter=reporter),
)
results = tuner.fit()

In [ ]:

manager_config = {
    "df": train_df,

}
config={
        "manager_config": manager_config
        }

env = HRL(config)°
env.observation_space.sample()

In [7]:
 
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hierarch_env", env_creator)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 

first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 
manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }


trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 5},local_dir="/Users/floriankockler/rayresults/training"),
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
    ),
 
    algorithm=a2c.A2C,
    config={
        "multiagent": {
            "policies": {
                "worker_policy": worker_policy_spec,
                "manager_policy": manager_policy_spec,
            },
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env": "hierarch_env",
        "env_config": hrl_config,
        "framework": "tf",
        "evaluation_num_workers": 1,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},
    },
)
result = trainer.fit()

2023-09-06 13:24:31,032	WARNING worker.py:2006 -- Warning: The actor AIRRLTrainer is very large (26 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=92326) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(AIRA2C pid=92326) 2023-09-06 13:24:38,323	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(AIRA2C pid=92326) 2023-09-06 13:24:38,323	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(AIRA2C pid=92326) 2023-09-06 13:24:38,323	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=92340) DeprecationWarning

TrainingFailedError: The Ray Train run failed. Please inspect the previous error messages for a cause. After fixing the issue (assuming that the error is not caused by your own application logic, but rather an error such as OOM), you can restart the run from scratch or continue this run.
To continue this run, you can use: `trainer = RLTrainer.restore("/Users/floriankockler/rayresults/training/AIRA2C_2023-09-06_13-24-30")`.
To start a new run that will retry on training failures, set `air.RunConfig(failure_config=air.FailureConfig(max_failures))` in the Trainer's `run_config` with `max_failures > 0`, or `max_failures = -1` for unlimited retries.

In [ ]:
# worker_config =   {
#         "cash_initial": 1000000, 
#         "trading_cost": 0.001, 
#         "tech_indicator_list": indicators,
#         "print_verbosity": 1,
#         "initial_shares_held": 0,
#         "df": df,  }


# env = Worker(env_config=worker_config)
# check_env(env)

In [ ]:


worker_config =   {
        "cash_initial": 1000000, 
        "trading_cost": 0.001, 
        "tech_indicator_list": indicators,
        "print_verbosity": 1,
        "initial_shares_held": 0,
        "df": train_df,  }


env = Worker(env_config=worker_config)

n_iterations = 30

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
import random

from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    # Assuming this is your environment
    return Worker(env_config)

register_env("Single_Stock", env_creator)


parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

worker_config =   {
        "cash_initial": 1000000, 
        "trading_cost": 0.001, 
        "tech_indicator_list": indicators,
        "print_verbosity": 1,
        "initial_shares_held": 0,
        "df": train_df,  
        }

def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    "rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(1000, 50000),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),
    "train_batch_size": tune.choice([200, 400, 600]),
}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "Single_Stock",
        "env_config": worker_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 5,  # 1 for training + 4 for sampling
        "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/tuning3", progress_reporter=reporter),
)
results = tuner.fit()